# This was Run on EULER as it needs a looot of memory (around 100GB RAM) should do

In [1]:
!pip install zenodo_get


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [4]:
!mkdir zips
!cd zips && python -m zenodo_get 5589195

mkdir: cannot create directory ‘zips’: File exists
Title: EDGAR-CORPUS
Keywords: SEC EDGAR 10-K Annual Reports, EDGAR, Electronic Data Gathering, Analysis, and Retrieval, 10-K, Annual Reports
Publication date: 2021-10-21
DOI: 10.5281/zenodo.5589195
Total size: 10938.6 MB

Link: https://zenodo.org/api/files/1fffb366-d027-4e20-bb70-8aafe307fdbf/1993.zip   size: 37.8 MB
100% [....................................................] 39671583 / 39671583
Checksum is correct. (c7a8c972a9d52a79df8def014c57d8a6)

Link: https://zenodo.org/api/files/1fffb366-d027-4e20-bb70-8aafe307fdbf/1994.zip   size: 68.7 MB
100% [....................................................] 72050322 / 72050322
Checksum is correct. (5158d44b8fde68cf8132d8d4c40ec2e4)

Link: https://zenodo.org/api/files/1fffb366-d027-4e20-bb70-8aafe307fdbf/1995.zip   size: 123.0 MB
100% [..................................................] 129020135 / 129020135
Checksum is correct. (0d27103775f4a0a28e892a61a5a25bf8)

Link: https://zenodo.org

In [1]:
from zipfile import ZipFile
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm
import json
import re

base_path = Path("/cluster/scratch/afelderer/zips")

In [2]:
insurance_4digit_sic_codes = ["6311", "6321","6324", "6331", 
                              "6351", "6361", "6399","6411"]

In [8]:
basepath = Path("/cluster/scratch/afelderer/zips")
    
df_complete = pd.DataFrame()
for year in tqdm(range(1993,2021)):
    zip_file = ZipFile(base_path/f"{year}.zip")
    print(f"Start: {year}")
    df = pd.DataFrame()
    for text_file in zip_file.infolist():
        if text_file.filename.endswith('.json'):
            df = pd.concat([df, pd.DataFrame({k:[v] for k,v in json.load(zip_file.open(text_file.filename)).items()})])
    df["year"] = year
    df_complete = pd.concat([df_complete, df])
    df.to_pickle(f"pickle/{year}.pkl")
    print(f"End: {year}")

  0%|          | 0/28 [00:00<?, ?it/s]

Start: 1993
End: 1993
Start: 1994
End: 1994
Start: 1995
End: 1995
Start: 1996
End: 1996
Start: 1997
End: 1997
Start: 1998
End: 1998
Start: 1999
End: 1999
Start: 2000
End: 2000
Start: 2001
End: 2001
Start: 2002
End: 2002
Start: 2003
End: 2003
Start: 2004
End: 2004
Start: 2005
End: 2005
Start: 2006
End: 2006
Start: 2007
End: 2007
Start: 2008
End: 2008
Start: 2009
End: 2009
Start: 2010
End: 2010
Start: 2011
End: 2011
Start: 2012
End: 2012
Start: 2013
End: 2013
Start: 2014
End: 2014
Start: 2015
End: 2015
Start: 2016
End: 2016
Start: 2017
End: 2017
Start: 2018
End: 2018
Start: 2019
End: 2019
Start: 2020
End: 2020


In [2]:
df_complete = pd.DataFrame()
for year in tqdm(range(1993, 2021)):
    df = pd.read_pickle(f"pickle/{year}.pkl")
    df["year"] = year
    df_complete = pd.concat([df_complete, df])

  0%|          | 0/28 [00:00<?, ?it/s]

In [4]:
print("Nr. of 10k filings:", df_complete.shape[0])
print("Nr. of legal entities:", len(df_complete.cik.unique()))
print("Nr. of industry segments:", len(df_complete.sic.unique()))

Nr. of 10k filings: 222901
Nr. of legal entities: 37797
Nr. of industry segments: 443


In [6]:
df = df_complete[df_complete["sic"].isin(insurance_4digit_sic_codes)]
df.reset_index(drop=True, inplace=True)
df.reset_index(inplace=True)
df = df.rename(columns={"index": "report_id"})
df.to_pickle(Path("/cluster/home/afelderer/Thesis")/"raw_10k.pkl")
df_complete.to_pickle(base_path/".."/"all_10k.pkl")
del df_complete

In [7]:
!rm -r zips
!rm -r pickle

## Inspecting duplicate entries (same cik and year)

- I did hand inspect double entries for the same company (cik) in one year. 

- Some of the doubles happen during the first year which could have something 
to do with the initial filing.

- Some are filed for part of the year and then the full year as well.

- Both the two above are kept as is.

- For some companies there is one report missing in a certain year while there are 
two in the following year and the first one is often filed early in that year. 
In these cases I adjust the year of the first report to the "missing year"

In [3]:
df = pd.read_pickle(Path("/cluster/home/afelderer/Thesis")/"raw_10k.pkl")

In [4]:
counts = df.groupby(["cik", "year"], as_index=False).size()
duplicates = counts[counts["size"] > 1]
df_duplicates = pd.DataFrame()
for index, row in duplicates.iterrows():
    df_duplicates = pd.concat([df_duplicates, df[(df["cik"] == row["cik"]) & (df["year"] == row["year"])][["cik", "year", "filing_date", "period_of_report","sic"]] ])
df_duplicates

,cik,year,filing_date,period_of_report,sic
453,100716,1996,1996-06-28,1996-03-31,6411
503,100716,1996,1997-03-31,1996-12-31,6331
4336,1017907,2011,2011-08-31,2011-06-30,6331
4343,1017907,2011,2012-02-29,2011-12-31,6331
676,1027287,1997,1997-06-27,1997-03-31,6411
857,1027287,1997,1998-03-30,1997-12-31,6411
2136,1063980,2003,2003-03-27,2003-03-27,6411
2279,1063980,2003,2004-03-12,2003-12-31,6411
2115,1075343,2003,2004-03-26,2003-12-31,6331
2252,1075343,2003,2003-03-31,2003-03-31,6331


In [5]:
df.drop(1006,inplace=True) # Seems to be an almost identical entry thus only keeping the newer filing id report_id = 943
df.loc[2993]["year"] = 2005 # From Manual inspection if a year is missing and the following has two filings this is adjusted
df.loc[4000]["year"] = 2008
df.loc[3425]["year"] = 2006
df.loc[4071]["year"] = 2008
df.loc[2762]["year"] = 2006
df.loc[4663]["year"] = 2011

/cluster/apps/nss/gcc-8.2.0/python/3.8.5/x86_64/lib64/python3.8/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [6]:
df_master = df[[x for x in df.columns if x[:4] != "item"]]
df_master.to_pickle(Path("/cluster/home/afelderer/Thesis")/"master_10k.pkl")

In [8]:
df = pd.wide_to_long(df,
                     stubnames='item_', 
                     i="report_id", 
                     j="item", 
                     suffix="\d+[AB]*").rename(columns={"item_": "raw_text"}).sort_index()

In [10]:
df[["raw_text"]].to_pickle(base_path/"raw_text.pkl")
df["text"] = df["raw_text"].apply(
    lambda x: re.sub(
        string=x, 
        pattern="ITEM\s\d+\.*", 
        repl="", 
        count=1, 
        flags=re.IGNORECASE).strip().split("\n")
    )
## Santiy Check
print("# Starting with item:", df[df["text"].apply(lambda x: x[0][:4] == "item")].shape[0])
print("# Starting with Item:",df[df["text"].apply(lambda x: x[0][:4] == "Item")].shape[0])
print("# Starting with ITEM:",df[df["text"].apply(lambda x: x[0][:4] == "ITEM")].shape[0])
df = df.drop("raw_text", axis=1)
df = df.explode("text")
df["n_words"] = df.text.str.split(" ").str.len()

# Starting with item: 0
# Starting with Item: 0
# Starting with ITEM: 0


In [22]:
#df.reset_index(inplace=True)
#df = df[["report_id", "item", "text", "n_words"]]
df["paragraph_nr"] = df.groupby(['report_id']).cumcount()
df["loss_kw"] = df["text"].apply(lambda x: "loss" in x.lower())
df["unexpected_kw"] = df["text"].apply(lambda x: "unexpected" in x.lower())
df.to_pickle(Path("/cluster/home/afelderer/Thesis")/"paragraphs_10k.pkl")

/scratch/259395716.tmpdir/ipykernel_126546/1021089180.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["paragraph_nr"] = df.groupby(['report_id']).cumcount()
/scratch/259395716.tmpdir/ipykernel_126546/1021089180.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["loss_kw"] = df["text"].apply(lambda x: "loss" in x.lower())
/scratch/259395716.tmpdir/ipykernel_126546/1021089180.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [20]:
relevant_sections = ["2", "6", "7", "8"]
df[df.item.isin(relevant_sections)].to_pickle(Path("/cluster/home/afelderer/Thesis")/"paragraphs_relevant_10k.pkl")

In [37]:
df.groupby(["loss_kw", "unexpected_kw"]).count()

report_id     item     text  n_words  paragraph_nr
loss_kw unexpected_kw                                                    
False   False            6696309  6696309  6696309  6696309       6696309
        True                3465     3465     3465     3465          3465
True    False             593930   593930   593930   593930        593930
        True                3013     3013     3013     3013          3013

In [35]:
df[df.item.isin(relevant_sections)].groupby(["loss_kw", "unexpected_kw"]).count()

report_id     item     text  n_words  paragraph_nr
loss_kw unexpected_kw                                                    
False   False            3010982  3010982  3010982  3010982       3010982
        True                 878      878      878      878           878
True    False             342728   342728   342728   342728        342728
        True                1157     1157     1157     1157          1157